# generate_error_table

## imports

In [2]:
import pandas as pd
import numpy as np

## load data

In [3]:
df0 = pd.read_csv('data/benchmarks_20230122.csv')
df1 = pd.read_csv('data/benchmarks_20230126.csv')
df2 = pd.read_csv('data/equiwidth_20230209.csv')
df3 = pd.read_csv('data/gdy_20230216.csv')
df = pd.concat([df0, df1, df2, df3], axis=0)
df = df.fillna('TPC-DS')

## generate errors

In [4]:
def generate_errors(df):
    df = df.copy()  # Convenience
    
    df['absolute_error'] = np.absolute(df['estimated_output'] - df['real_output'])
    df['relative_error'] = df['absolute_error'] / df['real_output']
    df['x'] = df['estimated_output'] / df['real_output']
    df['1/x'] = 1 / df['x']
    df['q_error'] = df[['x','1/x']].max(axis=1)
    df['real_output_lower_bound'] = df['real_output']
    df['real_output_lower_bound'] = df['real_output_lower_bound'].clip(1)
    df['estimated_output_lower_bound'] = df['estimated_output']
    df['estimated_output_lower_bound'] = df['estimated_output_lower_bound'].clip(1)
    df['pseudo_x'] = df['estimated_output_lower_bound'] / df['real_output_lower_bound']
    df['pseudo_1/x'] = 1 / df['pseudo_x']
    df['pseudo_q_error'] = df[['pseudo_x','pseudo_1/x']].max(axis=1)
    df['mean_squared_error'] = df['absolute_error'] ** 2
    df.drop(['x', '1/x', 'real_output_lower_bound', 'estimated_output_lower_bound', 'pseudo_x', 'pseudo_1/x'], axis=1, inplace = True)

    return df

## result

### with_NULLs

In [5]:
df_with_errors = generate_errors(df)
df_with_errors

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram             4.436990e+05   
          EquiHeightHistogram                     4.923148e+05   
          EquiWidthHistogram                      4.961650e+05   
          GDYHistogram                            4.921751e+05   
          MaxDiffFrequencyHistogram               4.983394e+05   
TPC-DS    EqualDistinctCountHistogram             1.627086e+05   
          EquiHeightHistogram                     1.599385e+05   
          EquiWidthHistogram                      1.660090e+05   
          GDYHistogram                            1.340737e+06   
          MaxDiffFrequencyHistogram               1.625543e+05   
TPC-H     EqualDistinctCountHistogram             9.039773e+03   
          EquiHeightHistogram                     8.745382e+03   
          EquiWidthHistogram                      9.033915e+03   
          GDYHistogram                            8.342200e+06   
          MaxDiffFrequencyHistogram               8.825051e+03   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram         1.162159e+05   
          EquiHeightHistogram                 1.180125e+05   
          EquiWidthHistogram                  1.180460e+05   
          GDYHistogram                        1.187373e+05   
          MaxDiffFrequencyHistogram           1.211411e+05   
TPC-DS    EqualDistinctCountHistogram         6.165372e+04   
          EquiHeightHistogram                 6.053633e+04   
          EquiWidthHistogram                  6.407025e+04   
          GDYHistogram                        5.587984e+05   
          MaxDiffFrequencyHistogram           6.162713e+04   
TPC-H     EqualDistinctCountHistogram         2.468485e+03   
          EquiHeightHistogram                 2.185476e+03   
          EquiWidthHistogram                  2.471828e+03   
          GDYHistogram                        1.847086e+06   
          MaxDiffFrequencyHistogram           2.357634e+03   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          GDYHistogram                                 NaN           NaN   
          MaxDiffFrequencyHistogram                    NaN           NaN   
TPC-DS    EqualDistinctCountHistogram            22.099158           NaN   
          EquiHeightHistogram                    22.099502           NaN   
          EquiWidthHistogram                     21.618539           NaN   
          GDYHistogram                                 NaN           NaN   
          MaxDiffFrequencyHistogram              22.076917           NaN   
TPC-H     EqualDistinctCountHistogram                  NaN           NaN   
          EquiHeightHistogram                          NaN           NaN   
          EquiWidthHistogram                           NaN           NaN   
          GDYHistogram                        31618.378633  31727.584442   
          MaxDiffFrequencyHistogram                    NaN           NaN   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         26461.034006  
          EquiHeightHistogram                 11692.064000  
          EquiWidthHistogram                  13092.666368  
          GDYHistogram                        12278.477865  
          MaxDiffFrequencyHistogram           11149.412992  
TPC-DS    EqualDistinctCountHistogram          4657.808599  
          EquiHeightHistogram                  4899.630984  
          EquiWidth

### without_NULLs

In [6]:
size_before = len(df)
df = df.drop(df[df.real_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'real_output' cell was zero.")
df = df.drop(df[df.estimated_output == 0].index)
print(f"Removed {len(df)-size_before} ({(size_before-len(df))/size_before:.2%}) rows because the 'estimated_output' cell was zero.")

df_with_errors = generate_errors(df)

df_with_errors.groupby([#"operator_type",
                        "benchmark",
                        "histogram"]).agg(root_mean_squared_error=("mean_squared_error", lambda x: np.sqrt(np.mean(x))),
                                          mean_absolute_error=("absolute_error", np.mean),
                                          mean_relative_error=("relative_error", np.mean),
                                          mean_q_error=("q_error", np.mean),
                                          mean_pseudo_q_error=("pseudo_q_error", np.mean))

Removed -331 (4.61%) rows because the 'real_output' cell was zero.
Removed -1233 (17.18%) rows because the 'estimated_output' cell was zero.


root_mean_squared_error  \
benchmark histogram                                              
JoinOrder EqualDistinctCountHistogram             4.694869e+05   
          EquiHeightHistogram                     5.191989e+05   
          EquiWidthHistogram                      5.211591e+05   
          GDYHistogram                            5.120312e+05   
          MaxDiffFrequencyHistogram               4.704292e+05   
TPC-DS    EqualDistinctCountHistogram             1.664812e+05   
          EquiHeightHistogram                     1.663866e+05   
          EquiWidthHistogram                      1.915863e+05   
          GDYHistogram                            1.364340e+06   
          MaxDiffFrequencyHistogram               1.657429e+05   
TPC-H     EqualDistinctCountHistogram             5.906786e+03   
          EquiHeightHistogram                     5.362748e+03   
          EquiWidthHistogram                      6.406539e+03   
          GDYHistogram                            9.373861e+06   
          MaxDiffFrequencyHistogram               6.027857e+03   

                                       mean_absolute_error  \
benchmark histogram                                          
JoinOrder EqualDistinctCountHistogram         1.269772e+05   
          EquiHeightHistogram                 1.276697e+05   
          EquiWidthHistogram                  1.304190e+05   
          GDYHistogram                        1.291280e+05   
          MaxDiffFrequencyHistogram           1.199132e+05   
TPC-DS    EqualDistinctCountHistogram         6.407457e+04   
          EquiHeightHistogram                 6.473453e+04   
          EquiWidthHistogram                  7.972532e+04   
          GDYHistogram                        5.882860e+05   
          MaxDiffFrequencyHistogram           6.389849e+04   
TPC-H     EqualDistinctCountHistogram         1.446530e+03   
          EquiHeightHistogram                 1.162721e+03   
          EquiWidthHistogram                  1.646114e+03   
          GDYHistogram                        2.116230e+06   
          MaxDiffFrequencyHistogram           1.535407e+03   

                                       mean_relative_error  mean_q_error  \
benchmark histogram                                                        
JoinOrder EqualDistinctCountHistogram           432.409415  2.711881e+06   
          EquiHeightHistogram                   614.642440  4.338387e+05   
          EquiWidthHistogram                    526.374420  2.710359e+05   
          GDYHistogram                          367.184259  3.000255e+05   
          MaxDiffFrequencyHistogram             595.349400  5.253562e+04   
TPC-DS    EqualDistinctCountHistogram            23.100213  2.819712e+08   
          EquiHeightHistogram                    23.851159  2.915432e+08   
          EquiWidthHistogram                     21.343049  2.649180e+04   
          GDYHistogram                           65.195507  4.128405e+09   
          MaxDiffFrequencyHistogram              22.916939  5.038439e+08   
TPC-H     EqualDistinctCountHistogram            40.997071  7.237080e+05   
          EquiHeightHistogram                    27.232247  1.146574e+06   
          EquiWidthHistogram                      0.917758  9.690899e+05   
          GDYHistogram                        39267.161316  3.941027e+04   
          MaxDiffFrequencyHistogram              32.115328  1.529549e+06   

                                       mean_pseudo_q_error  
benchmark histogram                                         
JoinOrder EqualDistinctCountHistogram         27346.490249  
          EquiHeightHistogram                 10712.631358  
          EquiWidthHistogram                  12356.022324  
          GDYHistogram                        11252.152216  
          MaxDiffFrequencyHistogram           10546.295897  
TPC-DS    EqualDistinctCountHistogram          4374.478350  
          EquiHeightHistogram                  4478.932424  
          EquiWidth

## compare_operators

In [6]:
grouped = df.groupby(["benchmark", "histogram", "query", "operator_type"]).size()

queries = df[["benchmark", "query"]].drop_duplicates()

# super inefficient, doesn't matter for now
op_counts = {}
for _, outer_benchmark, outer_query in queries.itertuples():
    for (_1, benchmark, histogram, query, operator_type, count) in grouped.reset_index().itertuples():
        if outer_benchmark == benchmark and outer_query == query:
            if (benchmark, query, operator_type) not in op_counts:
                op_counts[(benchmark, query, operator_type)] = count
                continue

            if op_counts[(benchmark, query, operator_type)] != count:
                print(f"WARNING: different value of {count} (previously {op_counts[(benchmark, query, operator_type)]}) for {benchmark}, query {query}, op {operator_type}")
